# Proyecto: Motor de Base de Datos Relacional basado en Teoría de Conjuntos
**Equipo**: Anderson Fabián González Aparicio
**Fecha**: [Fecha de entrega]

## Objetivo del Proyecto
Este proyecto implementa un motor de base de datos relacional utilizando exclusivamente operaciones de teoría de conjuntos. La implementación demuestra la estrecha relación entre la teoría de conjuntos y las bases de datos relacionales.

## Características Principales
1. Implementación desde cero de operaciones fundamentales
2. Manejo de tablas como conjuntos de tuplas
3. Operaciones relacionales basadas en teoría de conjuntos
4. Ejemplos prácticos de aplicación

## Guía del Notebook
1. Fundamentos Teóricos
2. Implementación del Motor
3. Ejemplos y Casos de Uso
4. Pruebas y Demostraciones

# Motor de Base de Datos Relacional basado en Teoría de Conjuntos

## 1. Fundamentos Teóricos

### 1.1 Teoría de Conjuntos y Bases de Datos Relacionales

La teoría de conjuntos proporciona el fundamento matemático esencial para las bases de datos relacionales. Esta relación se establece a través de varios conceptos fundamentales:

#### Conceptos Básicos de Teoría de Conjuntos:

1. **Definición de Conjunto**:
   - Un conjunto es una colección de elementos únicos y no ordenados
   - Notación: A = {a₁, a₂, ..., aₙ}
   - Propiedades:
     * No hay elementos duplicados
     * El orden no importa
     * Pueden ser finitos o infinitos

2. **Operaciones Fundamentales**:
   - **Unión (∪)**: A ∪ B = {x | x ∈ A ∨ x ∈ B}
   - **Intersección (∩)**: A ∩ B = {x | x ∈ A ∧ x ∈ B}
   - **Diferencia (-)**: A - B = {x | x ∈ A ∧ x ∉ B}
   - **Producto Cartesiano (×)**: A × B = {(a,b) | a ∈ A ∧ b ∈ B}

#### Relación con Bases de Datos:

1. **Tablas como Conjuntos**:
   - Cada tabla es un conjunto de tuplas (filas)
   - Cada tupla es un elemento único del conjunto
   - El orden de las tuplas no afecta la información

2. **Registros como Tuplas**:
   - Cada registro es una tupla ordenada de valores
   - La estructura de la tupla está definida por el esquema de la tabla
   - Los valores nulos representan información ausente

3. **Operaciones Relacionales**:
   - **Selección (σ)**: Subconjunto que cumple una condición
   - **Proyección (π)**: Subset de atributos
   - **Join (⋈)**: Combinación basada en condiciones

### 1.2 Implementación en Bases de Datos Relacionales

1. **Integridad de Datos**:
   - La unicidad de elementos garantiza la integridad
   - Las claves primarias aseguran tuplas únicas
   - Las relaciones se mantienen mediante operaciones de conjuntos

2. **Consultas y Operaciones**:
   - SELECT → Operación de selección (σ)
   - PROJECT → Operación de proyección (π)
   - JOIN → Combinación de conjuntos
   - UNION, INTERSECTION → Operaciones directas

3. **Ventajas del Modelo**:
   - Fundamento matemático sólido
   - Operaciones bien definidas
   - Garantía de consistencia
   - Flexibilidad en consultas

In [10]:
from typing import List, Set, Dict, Tuple, Any, Callable, Optional, Union
from dataclasses import dataclass, field
from collections import namedtuple, defaultdict
import itertools
from enum import Enum
import operator
from datetime import datetime
import functools
import time

In [11]:
@dataclass
class Table:
    """Representa una tabla en nuestra base de datos relacional.
    Implementa las operaciones fundamentales de la teoría de conjuntos."""
    name: str
    columns: List[str]
    data: Set[Tuple] = field(default_factory=set)

    def __post_init__(self):
        """Validación post-inicialización"""
        for tuple_data in self.data:
            if len(tuple_data) != len(self.columns):
                raise ValueError(f"La tupla {tuple_data} no tiene el número correcto de columnas")

    def add_row(self, row: Tuple):
        """Añade una nueva fila (tupla) a la tabla"""
        if len(row) != len(self.columns):
            raise ValueError(f"La fila debe tener {len(self.columns)} columnas")
        self.data.add(row)

    def __str__(self):
        """Representación en formato tabular"""
        result = [f"Tabla: {self.name}"]
        result.append(" | ".join(self.columns))
        result.append("-" * (sum(len(col) for col in self.columns) + 3 * (len(self.columns) - 1)))
        for row in self.data:
            result.append(" | ".join(str(item) for item in row))
        return "\n".join(result)

    def union(self, other: 'Table') -> 'Table':
        """Implementa la operación de unión de conjuntos (∪)"""
        if self.columns != other.columns:
            raise ValueError("Las tablas deben tener las mismas columnas para realizar la unión")
        return Table(
            name=f"{self.name}_union_{other.name}",
            columns=self.columns,
            data=self.data.union(other.data)
        )

    def intersection(self, other: 'Table') -> 'Table':
        """Implementa la operación de intersección de conjuntos (∩)"""
        if self.columns != other.columns:
            raise ValueError("Las tablas deben tener las mismas columnas para realizar la intersección")
        return Table(
            name=f"{self.name}_intersect_{other.name}",
            columns=self.columns,
            data=self.data.intersection(other.data)
        )

    def difference(self, other: 'Table') -> 'Table':
        """Implementa la operación de diferencia de conjuntos (-)"""
        if self.columns != other.columns:
            raise ValueError("Las tablas deben tener las mismas columnas para realizar la diferencia")
        return Table(
            name=f"{self.name}_diff_{other.name}",
            columns=self.columns,
            data=self.data.difference(other.data)
        )

    def cartesian_product(self, other: 'Table') -> 'Table':
        """Implementa el producto cartesiano de conjuntos (×)"""
        new_columns = self.columns + other.columns
        new_data = set()
        for row1 in self.data:
            for row2 in other.data:
                new_data.add(row1 + row2)
        return Table(
            name=f"{self.name}_product_{other.name}",
            columns=new_columns,
            data=new_data
        )

    def select(self, condition: Callable[[Tuple], bool]) -> 'Table':
        """Implementa la operación de selección (σ)"""
        return Table(
            name=f"{self.name}_select",
            columns=self.columns,
            data=set(row for row in self.data if condition(row))
        )

    def project(self, columns: List[str]) -> 'Table':
        """Implementa la operación de proyección (π)"""
        if not all(col in self.columns for col in columns):
            raise ValueError("Columnas no válidas para la proyección")
        indices = [self.columns.index(col) for col in columns]
        new_data = set()
        for row in self.data:
            new_row = tuple(row[i] for i in indices)
            new_data.add(new_row)
        return Table(
            name=f"{self.name}_project",
            columns=columns,
            data=new_data
        )

    def join(self, other: 'Table', common_columns: List[str]) -> 'Table':
        """Implementa la operación de join natural (⋈)"""
        if not all(col in self.columns and col in other.columns for col in common_columns):
            raise ValueError("Las columnas comunes deben existir en ambas tablas")

        self_indices = [self.columns.index(col) for col in common_columns]
        other_indices = [other.columns.index(col) for col in common_columns]
        new_columns = self.columns + [col for col in other.columns if col not in common_columns]

        new_data = set()
        for row1 in self.data:
            for row2 in other.data:
                if all(row1[i1] == row2[i2] for i1, i2 in zip(self_indices, other_indices)):
                    new_row = row1 + tuple(v for i, v in enumerate(row2)
                                         if other.columns[i] not in common_columns)
                    new_data.add(new_row)

        return Table(
            name=f"{self.name}_join_{other.name}",
            columns=new_columns,
            data=new_data
        )

In [17]:
# Crear tablas de ejemplo
estudiantes = Table(
    name="Estudiantes",
    columns=["ID", "Nombre", "Edad", "Carrera"],
    data={
        (1, "Ana García", 20, "Informática"),
        (2, "Carlos López", 22, "Matemáticas"),
        (3, "María Rodríguez", 21, "Informática"),
        (4, "Juan Pérez", 23, "Física")
    }
)

estudiantes2 = Table(
    name="Estudiantes2",
    columns=["ID", "Nombre", "Edad", "Carrera"],
    data={
        (3, "María Rodríguez", 21, "Informática"),
        (4, "Juan Pérez", 23, "Física"),
        (5, "Laura Martínez", 20, "Matemáticas")
    }
)

cursos = Table(
    name="Cursos",
    columns=["CursoID", "Nombre", "Creditos", "Departamento"],
    data={
        (101, "Programación I", 4, "Informática"),
        (102, "Cálculo I", 5, "Matemáticas"),
        (103, "Física Básica", 4, "Física"),
        (104, "Estructuras Discretas", 4, "Matemáticas")
    }
)

inscripciones = Table(
    name="Inscripciones",
    columns=["ID", "CursoID", "Semestre", "Calificacion"],  # Cambiado EstudianteID por ID
    data={
        (1, 101, "2024-1", 85),
        (1, 102, "2024-1", 78),
        (2, 102, "2024-1", 92),
        (3, 101, "2024-1", 88),
        (4, 103, "2024-1", 95)
    }
)

# Mostrar tablas originales
print("TABLAS ORIGINALES:")
print("\nEstudiantes:")
print(estudiantes)
print("\nEstudiantes2:")
print(estudiantes2)
print("\nCursos:")
print(cursos)
print("\nInscripciones:")
print(inscripciones)

TABLAS ORIGINALES:

Estudiantes:
Tabla: Estudiantes
ID | Nombre | Edad | Carrera
----------------------------
2 | Carlos López | 22 | Matemáticas
3 | María Rodríguez | 21 | Informática
4 | Juan Pérez | 23 | Física
1 | Ana García | 20 | Informática

Estudiantes2:
Tabla: Estudiantes2
ID | Nombre | Edad | Carrera
----------------------------
4 | Juan Pérez | 23 | Física
3 | María Rodríguez | 21 | Informática
5 | Laura Martínez | 20 | Matemáticas

Cursos:
Tabla: Cursos
CursoID | Nombre | Creditos | Departamento
------------------------------------------
101 | Programación I | 4 | Informática
104 | Estructuras Discretas | 4 | Matemáticas
103 | Física Básica | 4 | Física
102 | Cálculo I | 5 | Matemáticas

Inscripciones:
Tabla: Inscripciones
ID | CursoID | Semestre | Calificacion
--------------------------------------
1 | 102 | 2024-1 | 78
1 | 101 | 2024-1 | 85
2 | 102 | 2024-1 | 92
4 | 103 | 2024-1 | 95
3 | 101 | 2024-1 | 88


In [19]:
# 1. Operaciones Fundamentales
print("\nOPERACIONES FUNDAMENTALES:")

# Unión
print("\n1.1 Unión de estudiantes:")
union_result = estudiantes.union(estudiantes2)
print(union_result)

# Intersección
print("\n1.2 Intersección de estudiantes:")
intersection_result = estudiantes.intersection(estudiantes2)
print(intersection_result)

# Diferencia
print("\n1.3 Diferencia de estudiantes:")
difference_result = estudiantes.difference(estudiantes2)
print(difference_result)

# Producto Cartesiano
print("\n1.4 Producto Cartesiano (estudiantes × cursos):")
cartesian_result = estudiantes.cartesian_product(cursos)
print(cartesian_result)

# 2. Operaciones Derivadas
print("\nOPERACIONES DERIVADAS:")

# Selección
print("\n2.1 Selección: Estudiantes de Informática mayores de 20 años:")
info_mayores_20 = estudiantes.select(
    lambda row: row[2] > 20 and row[3] == "Informática"
)
print(info_mayores_20)

# Proyección
print("\n2.2 Proyección: Nombres y carreras de estudiantes:")
nombres_carreras = estudiantes.project(["Nombre", "Carrera"])
print(nombres_carreras)

# Join simple
print("\n2.3 Join: Estudiantes con sus inscripciones:")
estudiantes_inscritos = estudiantes.join(inscripciones, ["ID"])
print(estudiantes_inscritos)

# Consulta compleja
print("\n3. Consulta Compleja:")
print("Estudiantes de Informática con sus cursos y calificaciones:")

# 1. Primero seleccionamos estudiantes de Informática
estudiantes_info = estudiantes.select(lambda row: row[3] == "Informática")

# 2. Hacemos join con inscripciones
estudiantes_info_cursos = estudiantes_info.join(inscripciones, ["ID"])

# 3. Hacemos join con la información de los cursos
resultado_final = estudiantes_info_cursos.join(cursos, ["CursoID"])

# 4. Proyectamos las columnas relevantes
columnas_deseadas = ["Nombre", "Carrera", "Calificacion"]  # Simplificado a columnas que sabemos que existen
resultado_final = resultado_final.project(columnas_deseadas)

print(resultado_final)

# Ejemplos adicionales de consultas
print("\n4. Ejemplos Adicionales:")

# Estudiantes por carrera
print("\n4.1 Número de estudiantes por carrera:")
carreras = estudiantes.project(["Carrera"])
print(carreras)

# Estudiantes con calificaciones mayores a 90
print("\n4.2 Estudiantes con calificaciones sobresalientes (>90):")
buenos_estudiantes = estudiantes.join(inscripciones, ["ID"]).select(
    lambda row: row[-1] > 90  # La calificación es el último campo
)
print(buenos_estudiantes)


OPERACIONES FUNDAMENTALES:

1.1 Unión de estudiantes:
Tabla: Estudiantes_union_Estudiantes2
ID | Nombre | Edad | Carrera
----------------------------
2 | Carlos López | 22 | Matemáticas
5 | Laura Martínez | 20 | Matemáticas
1 | Ana García | 20 | Informática
4 | Juan Pérez | 23 | Física
3 | María Rodríguez | 21 | Informática

1.2 Intersección de estudiantes:
Tabla: Estudiantes_intersect_Estudiantes2
ID | Nombre | Edad | Carrera
----------------------------
4 | Juan Pérez | 23 | Física
3 | María Rodríguez | 21 | Informática

1.3 Diferencia de estudiantes:
Tabla: Estudiantes_diff_Estudiantes2
ID | Nombre | Edad | Carrera
----------------------------
2 | Carlos López | 22 | Matemáticas
1 | Ana García | 20 | Informática

1.4 Producto Cartesiano (estudiantes × cursos):
Tabla: Estudiantes_product_Cursos
ID | Nombre | Edad | Carrera | CursoID | Nombre | Creditos | Departamento
-------------------------------------------------------------------------
4 | Juan Pérez | 23 | Física | 104 | Estruc

## 4. Documentación del Motor de Base de Datos

### 4.1 Estructura del Sistema
El motor implementa una base de datos relacional utilizando la teoría de conjuntos como fundamento matemático. La estructura principal es la clase `Table`, que representa una tabla como un conjunto de tuplas.

### 4.2 Operaciones Implementadas

#### 4.2.1 Operaciones Fundamentales
1. **Unión (∪)**
   - Combina registros de dos tablas compatibles
   - Elimina duplicados automáticamente
   - Requiere esquemas idénticos

2. **Intersección (∩)**
   - Encuentra registros comunes entre tablas
   - Mantiene solo tuplas presentes en ambas tablas
   - Requiere esquemas idénticos

3. **Diferencia (-)**
   - Obtiene registros únicos de una tabla
   - Elimina los registros presentes en la segunda tabla
   - Requiere esquemas idénticos

4. **Producto Cartesiano (×)**
   - Genera todas las combinaciones posibles
   - Base para operaciones de join
   - Combina esquemas de ambas tablas

#### 4.2.2 Operaciones Derivadas
1. **Selección (σ)**
   - Filtra registros según una condición
   - Mantiene el esquema original
   - Permite condiciones complejas

2. **Proyección (π)**
   - Selecciona columnas específicas
   - Puede reducir duplicados
   - Modifica el esquema

3. **Join (⋈)**
   - Combina tablas según columnas comunes
   - Mantiene la integridad referencial
   - Permite relaciones complejas

### 4.3 Uso y Mejores Prácticas
1. Validar compatibilidad de esquemas antes de operaciones
2. Usar condiciones de selección específicas
3. Considerar el rendimiento con grandes conjuntos de datos
4. Mantener la integridad referencial en joins

In [22]:
print("CASOS DE USO PRÁCTICOS")
print("\n1. Gestión Académica:")

# Caso 1: Encontrar estudiantes que están cursando materias de su propia carrera
print("\nEstudiantes cursando materias de su departamento:")
estudiantes_dep = estudiantes.join(inscripciones, ["ID"])
estudiantes_dep = estudiantes_dep.join(cursos, ["CursoID"])

# Imprimir las columnas disponibles para debug
print("Columnas disponibles:", estudiantes_dep.columns)

estudiantes_misma_carrera = estudiantes_dep.select(
    lambda row: row[3] == row[-1]  # Comparar Carrera con Departamento
)
# Usamos los nombres exactos de las columnas que existen después de los joins
print(estudiantes_misma_carrera.project(["Nombre", "Carrera", "Nombre"]))

# Caso 2: Estudiantes sin inscripciones (corregido)
print("\nEstudiantes sin inscripciones:")
estudiantes_con_inscripcion = estudiantes.join(inscripciones, ["ID"])
estudiantes_sin_inscripcion = estudiantes.difference(
    estudiantes_con_inscripcion.project(estudiantes.columns)
)
print(estudiantes_sin_inscripcion)

# Caso 3: Análisis de carga académica (corregido)
print("\nCarga académica por estudiante:")
carga_academica = estudiantes.join(inscripciones, ["ID"])
carga_academica = carga_academica.join(cursos, ["CursoID"])
print(carga_academica.project(["Nombre", "Creditos", "Semestre"]))

CASOS DE USO PRÁCTICOS

1. Gestión Académica:

Estudiantes cursando materias de su departamento:
Columnas disponibles: ['ID', 'Nombre', 'Edad', 'Carrera', 'CursoID', 'Semestre', 'Calificacion', 'Nombre', 'Creditos', 'Departamento']
Tabla: Estudiantes_join_Inscripciones_join_Cursos_select_project
Nombre | Carrera | Nombre
-------------------------
Ana García | Informática | Ana García
Juan Pérez | Física | Juan Pérez
Carlos López | Matemáticas | Carlos López
María Rodríguez | Informática | María Rodríguez

Estudiantes sin inscripciones:
Tabla: Estudiantes_diff_Estudiantes_join_Inscripciones_project
ID | Nombre | Edad | Carrera
----------------------------

Carga académica por estudiante:
Tabla: Estudiantes_join_Inscripciones_join_Cursos_project
Nombre | Creditos | Semestre
----------------------------
María Rodríguez | 4 | 2024-1
Carlos López | 5 | 2024-1
Ana García | 4 | 2024-1
Ana García | 5 | 2024-1
Juan Pérez | 4 | 2024-1


## 5. Análisis de Casos de Uso

### 5.1 Gestión Académica
Los casos anteriores demuestran cómo el motor puede manejar situaciones reales:

1. **Seguimiento de Especialización**
   - Identifica estudiantes cursando materias de su carrera
   - Ayuda en la planificación académica
   - Verifica coherencia en las inscripciones

2. **Control de Inscripciones**
   - Detecta estudiantes sin materias inscritas
   - Facilita el seguimiento administrativo
   - Permite acciones preventivas

3. **Análisis de Carga Académica**
   - Muestra distribución de créditos
   - Ayuda en la planificación de recursos
   - Identifica posibles sobrecargas

### 5.2 Ventajas del Enfoque Basado en Conjuntos
1. **Integridad de Datos**
   - Eliminación natural de duplicados
   - Consistencia en las operaciones
   - Mantenimiento de relaciones

2. **Flexibilidad en Consultas**
   - Combinación de operaciones básicas
   - Construcción de consultas complejas
   - Adaptabilidad a diferentes casos

3. **Eficiencia Conceptual**
   - Operaciones bien definidas
   - Base matemática sólida
   - Resultados predecibles

In [21]:
import time

def medir_tiempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fin = time.time()
        print(f"Tiempo de ejecución: {(fin - inicio)*1000:.2f} ms")
        return resultado
    return wrapper

# Pruebas de rendimiento
print("PRUEBAS DE RENDIMIENTO")

@medir_tiempo
def prueba_union():
    return estudiantes.union(estudiantes2)

@medir_tiempo
def prueba_join():
    return estudiantes.join(inscripciones, ["ID"])

@medir_tiempo
def prueba_consulta_compleja():
    est = estudiantes.join(inscripciones, ["ID"])
    est = est.join(cursos, ["CursoID"])
    return est.project(["Nombre", "Carrera", "Calificacion"])

print("\n1. Prueba de unión:")
resultado_union = prueba_union()

print("\n2. Prueba de join:")
resultado_join = prueba_join()

print("\n3. Prueba de consulta compleja:")
resultado_complejo = prueba_consulta_compleja()

PRUEBAS DE RENDIMIENTO

1. Prueba de unión:
Tiempo de ejecución: 0.02 ms

2. Prueba de join:
Tiempo de ejecución: 0.07 ms

3. Prueba de consulta compleja:
Tiempo de ejecución: 0.14 ms


## 6. Limitaciones y Consideraciones

### 6.1 Limitaciones Técnicas

1. **Manejo de Nombres de Columnas**:
   - Los joins pueden crear ambigüedad en nombres de columnas
   - No hay renombramiento automático de columnas duplicadas
   - Se requiere cuidado especial al proyectar columnas después de joins

2. **Eficiencia**:
   - El uso de conjuntos (sets) puede ser ineficiente para grandes volúmenes de datos
   - Las operaciones de producto cartesiano pueden consumir mucha memoria
   - No hay optimización de consultas

3. **Tipos de Datos**:
   - No hay manejo específico de tipos de datos (todo se trata como strings o números)
   - No hay validación de tipos en las operaciones
   - Falta soporte para NULL o valores nulos

### 6.2 Limitaciones Funcionales

1. **Operaciones Ausentes**:
   - No hay soporte para agregaciones (COUNT, SUM, AVG, etc.)
   - Falta implementación de ORDER BY o clasificación
   - No hay soporte para subconsultas complejas

2. **Integridad Referencial**:
   - No hay manejo automático de claves foráneas
   - No se verifican restricciones de integridad
   - Falta manejo de cascada en operaciones

3. **Persistencia**:
   - Los datos solo existen en memoria
   - No hay almacenamiento permanente
   - Falta sistema de recuperación

### 6.3 Mejoras Potenciales

1. **Optimizaciones Posibles**:
   - Implementar índices para búsquedas más rápidas
   - Agregar cache para operaciones frecuentes
   - Optimizar algoritmos de join

2. **Funcionalidades Adicionales**:
   - Agregar soporte para funciones de agregación
   - Implementar ordenamiento de resultados
   - Añadir manejo de transacciones

3. **Mejoras de Usabilidad**:
   - Crear una interfaz más amigable para consultas
   - Implementar un parser SQL simple
   - Agregar validación de datos más robusta